# Gauss elimination and LU decomposition

In [1]:
import numpy as np

### Utility

In [2]:
def generate_matrix(n_cols: int, n_rows: int):
    return np.random.uniform(low=0.1, high=0.01, size=(n_rows, n_cols))

def generate_vector(n: int):
    return np.random.uniform(low=0.1, high=0.01, size=(n,))

### Gauss elimination

Without pivoting

In [3]:
def gauss_elimination(A, b):
    n = len(A)
    
    for i in range(n):
        divisor = A[i, i]
        A[i] /= divisor
        b[i] /= divisor
        
        for j in range(i + 1, n):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]

    for i in range(n - 1, -1, -1):
        for j in range(i):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]
    
    return b


With pivoting

In [4]:
def gauss_elimination_pivoting(A, b):
    n = len(A)
    pivots = []
    
    for i in range(n):
        pivot_row = np.argmax(np.abs(A[i:, i])) + i
        if pivot_row != i:
            pivots.append((i, pivot_row))
            A[[i, pivot_row]] = A[[pivot_row, i]]
            b[[i, pivot_row]] = b[[pivot_row, i]]
        
        divisor = A[i, i]
        A[i] /= divisor
        b[i] /= divisor
        
        for j in range(i + 1, n):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]

    for i in range(n - 1, -1, -1):
        for j in range(i):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]

    for i, pivot_row in pivots[::-1]: 
        A[[i, pivot_row]] = A[[pivot_row, i]]
        b[[i, pivot_row]] = b[[pivot_row, i]]

    return b

In [5]:
from numpy.linalg import solve

n = 17 + 9

A = generate_matrix(n, n)
b = generate_vector(n)

result_gauss = gauss_elimination(A, b)
result_gauss_pivot = gauss_elimination_pivoting(A, b)
result_numpy = solve(A, b)

print(result_gauss)
print(result_gauss_pivot)
print(result_numpy)

print(f"Are the results close: {np.allclose(result_gauss, result_numpy)}")
print(f"Are the results close: {np.allclose(result_gauss_pivot, result_numpy)}")

[ 2.83798919  1.80229914  5.36850784 -1.30144054  0.64033313  0.42675393
 -1.41472134 -0.6503081  -4.08639575  1.07940725  1.76257729 -0.62767064
 -0.77051725  0.37871435  0.87425327 -2.02923471 -5.4361172  -0.6154549
  2.529499   -3.97580059 -1.30313638  1.14454921  3.20074878  2.80350471
  0.01046537 -0.62280486]
[ 2.83798919  1.80229914  5.36850784 -1.30144054  0.64033313  0.42675393
 -1.41472134 -0.6503081  -4.08639575  1.07940725  1.76257729 -0.62767064
 -0.77051725  0.37871435  0.87425327 -2.02923471 -5.4361172  -0.6154549
  2.529499   -3.97580059 -1.30313638  1.14454921  3.20074878  2.80350471
  0.01046537 -0.62280486]
[ 2.83798919  1.80229914  5.36850784 -1.30144054  0.64033313  0.42675393
 -1.41472134 -0.6503081  -4.08639575  1.07940725  1.76257729 -0.62767064
 -0.77051725  0.37871435  0.87425327 -2.02923471 -5.4361172  -0.6154549
  2.529499   -3.97580059 -1.30313638  1.14454921  3.20074878  2.80350471
  0.01046537 -0.62280486]
Are the results close: True
Are the results close

### LU decomposition

With pivoting

In [6]:
def lu_decomposition(A):
    n = len(A)
    L = np.eye(n)
    U = A.copy()

    for i in range(n):
        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] -= L[j, i] * U[i, i:]

    return L, U


With pivoting

In [7]:
def lu_decomposition_pivoting(A):
    n = len(A)
    L = np.zeros((n, n))
    U = A.copy()
    P = np.eye(n)

    for i in range(n):
        pivot_row = np.argmax(np.abs(U[i:, i])) + i
        if pivot_row != i:
            U[[i, pivot_row]] = U[[pivot_row, i]]
            L[[i, pivot_row]] = L[[pivot_row, i]]
            P[[i, pivot_row]] = P[[pivot_row, i]]

        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] -= L[j, i] * U[i, i:]

    L = np.eye(n) + L
    return P.T, L, U


A = generate_matrix(3, 3)

L, U = lu_decomposition(A)
print(np.allclose(L @ U, A))

P, L, U = lu_decomposition_pivoting(A)
print(np.allclose(P @ L @ U, A))

True
True


In [8]:
from scipy.linalg import lu

n = 17 + 9
n = 3

A = generate_matrix(n, n)

P, L, U = lu_decomposition_pivoting(A)
sp_P, sp_L, sp_U = lu(A)


print(L)
print(sp_L)
print(f"Are the results close: {np.allclose(L, sp_L)}, {np.allclose(U, sp_U)}")

[[1.         0.         0.        ]
 [0.42888764 1.         0.        ]
 [0.40078382 0.24292223 1.        ]]
[[1.         0.         0.        ]
 [0.42888764 1.         0.        ]
 [0.40078382 0.24292223 1.        ]]
Are the results close: True, True
